<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Selenium" data-toc-modified-id="Selenium-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Selenium</a></span><ul class="toc-item"><li><span><a href="#Los-drivers" data-toc-modified-id="Los-drivers-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Los drivers</a></span></li><li><span><a href="#Capturar-elementos-con-Selenium" data-toc-modified-id="Capturar-elementos-con-Selenium-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Capturar elementos con Selenium</a></span></li></ul></li><li><span><a href="#Manos-a-la-obra" data-toc-modified-id="Manos-a-la-obra-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Manos a la obra</a></span></li></ul></div>

# Selenium

Hacer web scraping se ha convertido en una herramienta muy poderosa para la extracción de datos y con ellos alimentar nuestras aplicaciones para realizar tareas increíbles. En esta ocasión usaremos ese poder para obtener las recompensas del mundial de League of Legends.
Haremos uso de selenium pues nos permite de manera muy dinámica explorar y obtener elementos de árbol de HTML e interactuar con los resultados de las acciones.

Antes de nada nos instalaremos selenium

```
pip install selenium
```

In [1]:
#!pip install selenium

## Los drivers

Selenium emplea “drivers” para poder acceder al contenido del sitio web, un driver es solo una versión reducida de un navegador que el código de selenium  utiliza, existen por ejemplo los drivers. Para descargarse los drivers: 


- [Driver Chrome](https://chromedriver.storage.googleapis.com/index.html?path=2.43/)


- [Driver Firefox](https://github.com/mozilla/geckodriver/releases)


- [Driver Opera](https://github.com/operasoftware/operachromiumdriver/releases)


> Cuando nos hayamos descargado el driver, tendremos un archivo .exe que tendremos que guardar en la misma carpeta de donde estemos trabajando. 

Podemos cargar los `drivers` de dos formas: 

```python
driver = webdriver.Chrome("/RutaAlDriver/chromedriver.exe")
```


También lo podemos hacer sin necesidad de descargarnos el driver: 

```python
driver = webdriver.Chrome(ChromeDriverManager().install())
```

## Capturar elementos con Selenium 

- Para encontrar un elemento 

```python
find_element_by_id
find_element_by_name
find_element_by_xpath
find_element_by_link_text
find_element_by_partial_link_text
find_element_by_tag_name
find_element_by_class_name
find_element_by_css_selector
```
- Para encontrar múltiples elementos (este método nos devuelve una lista):

```python
find_elements_by_name
find_elements_by_xpath
find_elements_by_link_text
find_elements_by_partial_link_text
find_elements_by_tag_name
find_elements_by_class_name
find_elements_by_css_selector
```

- Otros mátodos que pueden ser interesantes: 

```python
.click() # simula un click en el navegador
.send_keys # va a rellenar un campo
.implicitly_wait # # indica al Selenium WebDriver que espere un determinado tiempo antes de lanzar una excepción. Una vez que se establece este tiempo, WebDriver esperará el elemento antes de que se produzca la excepción.
.text # obtener el texto del elemento selenio
```

In [2]:
import requests
import pandas as pd
from time import sleep
import numpy as np


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys



import warnings
warnings.filterwarnings('ignore')


# Manos a la obra 

Lo primero que vamos a hacer es definir una serie de opciones para trabajar con Selenium

In [3]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot, pero me sigue diciendo que hay un software automatico usando chrome
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [4]:
# iniciamos el driver
driver = webdriver.Chrome(ChromeDriverManager().install())

## accedemos a la pagina web
driver.get('https://www.wunderground.com/history')

# esperamos
driver.implicitly_wait(5)

# aceptamos las cookies
driver.find_element_by_css_selector('#truste-consent-button').click()
sleep(5)

# seleccionamos la casilla para poner la ciudad por la que queremos buscar
ciudad = driver.find_element_by_css_selector('#historySearch').send_keys('Madrid, Madrid, Spain', Keys.TAB)
print("llegamos")
sleep(5)

# por lo que sea le tenemos que dar dos veces al botón de enviar
print("esperando1")
driver.find_element_by_xpath('//*[@id="dateSubmit"]').click()
sleep(5)
print("esperando2")
driver.find_element_by_xpath('//*[@id="dateSubmit"]').click()

# seleccionamos que nos los valores medios del mes
sleep(5)
driver.find_element_by_css_selector("#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)").click()




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/anagarciagarcia/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


llegamos
esperando1
esperando2


Hemos accedido a la información de un dia, perfecto! Pero que pasa si queremos más dias? 


In [5]:
url_list = []
for year in range(2010, 2022):
    for month in range(1,13):
        url_list.append(f"https://www.wunderground.com/history/monthly/LEMD/date/{year}-{month}")
    

In [6]:
# ya tenemos la lista de las url que queremos. 

url_list[:2]

['https://www.wunderground.com/history/monthly/LEMD/date/2010-1',
 'https://www.wunderground.com/history/monthly/LEMD/date/2010-2']

In [7]:
len(url_list)

144

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())
result_list = []
for i in url_list[:3]:
    driver.get(i)
    try: 
        sleep(5)
        # aceptamos cookies
        driver.find_element_by_css_selector('#truste-consent-button').click()
        driver.implicitly_wait(5)
        # sacamos el texto de la tabla
        my_info = driver.find_element_by_css_selector("#inner-content > div.region-content-main > div.row > div:nth-child(5) > div:nth-child(1) > div > lib-city-history-observation > div > div.observation-table.ng-star-inserted").text
        result_list.append(my_info)
    except:
        sleep(5)
        my_info = driver.find_element_by_css_selector("#inner-content > div.region-content-main > div.row > div:nth-child(5) > div:nth-child(1) > div > lib-city-history-observation > div > div.observation-table.ng-star-inserted").text
        result_list.append(my_info)

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/anagarciagarcia/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [9]:
# cada uno de los elementos de nuestra lista será una tabla

result_list[2]

'Time Temperature (° F) Dew Point (° F) Humidity (%) Wind Speed (mph) Pressure (Hg) Precipitation (in)\nMar\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\n18\n19\n20\n21\n22\n23\n24\n25\n26\n27\n28\n29\n30\n31\nMax Avg Min\n57 49.2 45\n61 48.7 37\n57 49.4 45\n59 49.6 39\n52 45.5 41\n43 39.0 36\n50 43.7 39\n41 36.9 32\n50 38.4 28\n46 37.7 28\n46 37.5 27\n45 39.8 36\n52 39.6 27\n59 45.0 32\n61 44.8 28\n63 47.1 32\n66 51.8 36\n64 50.9 41\n61 52.7 45\n66 57.8 50\n61 54.6 48\n68 55.4 43\n66 54.7 43\n66 57.3 48\n55 50.5 46\n55 47.9 41\n63 53.3 45\n68 53.6 41\n59 53.0 45\n57 50.8 45\n59 51.1 45\nMax Avg Min\n45 41.5 39\n45 29.6 0\n46 44.5 41\n48 43.3 39\n46 41.0 37\n39 37.1 36\n45 40.9 37\n37 21.6 0\n25 20.2 14\n25 20.0 18\n27 23.8 21\n30 27.9 27\n28 26.0 23\n30 25.0 16\n28 22.9 9\n34 24.9 14\n34 22.4 0\n45 38.5 0\n54 47.7 41\n52 46.4 39\n52 48.5 46\n50 45.1 39\n50 43.5 39\n54 48.5 45\n50 44.3 39\n43 37.4 0\n45 33.9 0\n43 38.2 34\n50 43.8 41\n50 40.2 34\n41 21.5 0\nMax Avg Min\n9

Convertimos toda la lista en dataframe

In [10]:
df = pd.DataFrame(result_list)
df

,0
0,Time Temperature (° F) Dew Point (° F) Humidit...
1,Time Temperature (° F) Dew Point (° F) Humidit...
2,Time Temperature (° F) Dew Point (° F) Humidit...


In [11]:
# sacamos la primera fila para ver como tenemos la info en cada fila. 

x = list(df.iloc[0][0].split("\n"))

In [12]:
# qué es x?

x[0]

'Time Temperature (° F) Dew Point (° F) Humidity (%) Wind Speed (mph) Pressure (Hg) Precipitation (in)'

In [13]:
# lo podemos convertir a dataframe?

pd.DataFrame(np.array(x[1:]).reshape(7, 32)).T

,0,1,2,3,4,5,6
0,Jan,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Total
1,1,48 42.9 39,41 36.2 34,81 77.0 66,29 16.6 7,28.0 27.8 27.7,0.00
2,2,46 41.7 37,41 35.8 0,100 84.0 66,10 3.9 0,28.1 28.1 28.0,0.00
3,3,46 42.3 36,43 39.3 36,100 89.7 76,7 1.5 0,28.1 28.0 28.0,0.00
4,4,50 46.6 45,48 44.9 43,100 94.0 87,8 4.3 0,28.0 27.7 27.6,0.00
5,5,52 46.8 43,46 44.1 39,100 88.9 76,5 1.7 0,27.6 27.6 27.5,0.00
6,6,46 42.4 39,39 23.0 0,93 70.9 53,15 7.3 0,27.7 27.7 27.6,0.00
7,7,39 36.3 34,37 22.8 0,93 87.0 65,23 11.7 0,27.6 27.5 27.5,0.00
8,8,37 34.3 32,23 19.0 16,65 54.4 44,28 18.5 7,27.9 27.7 27.6,0.00
9,9,36 32.8 30,21 18.1 14,64 55.4 47,20 7.2 0,28.0 27.9 27.9,0.00


Para hacer el `reshape` necesitamos tener en cuenta el número de dias que hay en cada mes. Lo que podemos hacer es sacar el índice del valor `"Max Avg Min"` que será el siguiente valor después de terminar la información de los dias. 

**¿Nos acordamos del método `index`?** 👇🏽

In [14]:
x

['Time Temperature (° F) Dew Point (° F) Humidity (%) Wind Speed (mph) Pressure (Hg) Precipitation (in)',
 'Jan',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 'Max Avg Min',
 '48 42.9 39',
 '46 41.7 37',
 '46 42.3 36',
 '50 46.6 45',
 '52 46.8 43',
 '46 42.4 39',
 '39 36.3 34',
 '37 34.3 32',
 '36 32.8 30',
 '32 26.9 21',
 '34 27.5 23',
 '45 16.9 0',
 '46 40.8 36',
 '52 49.5 45',
 '48 41.7 34',
 '50 44.1 39',
 '50 46.5 45',
 '59 51.5 46',
 '55 52.8 50',
 '57 47.7 37',
 '52 41.4 32',
 '50 42.0 34',
 '54 47.5 43',
 '55 47.4 41',
 '48 42.9 37',
 '50 38.1 0',
 '43 37.4 34',
 '54 37.5 25',
 '55 42.0 28',
 '52 44.3 32',
 '52 45.7 36',
 'Max Avg Min',
 '41 36.2 34',
 '41 35.8 0',
 '43 39.3 36',
 '48 44.9 43',
 '46 44.1 39',
 '39 23.0 0',
 '37 22.8 0',
 '23 19.0 16',
 '21 18.1 14',
 '27 21.0 18',
 '30 25.3 21',
 '36 16.2 0',
 '45 38

In [15]:
# nos devuelve la posición en nuestro dataframe. 

x[1:].index("Max Avg Min")

32

In [16]:
x[1]

'Jan'

In [17]:
clima_df = pd.DataFrame()
for i in range(len(df)):
    x = list(df.iloc[i][0].split("\n")) # "i" será cada fila de nuestro dataframe
    
    mi_ind = x[1:].index("Max Avg Min") # sacamos la posicion de "Max Avg Min" para hacer el reshape
    print(x[1])
    df_solo = pd.DataFrame(np.array(x[1:]).reshape(7, mi_ind)).T
    df_solo.columns = ["Time", "Temperature (° F)", "Dew Point (° F)", "Humidity (%)", "Wind Speed (mph)", "Pressure (Hg)", "Precipitation (in)"]
    df_solo["month"] = x[1]
    clima_df = pd.concat( [clima_df, df_solo], axis = 0)

Jan
Feb
Mar


In [18]:
clima_df.head(34)

,Time,Temperature (° F),Dew Point (° F),Humidity (%),Wind Speed (mph),Pressure (Hg),Precipitation (in),month
0,Jan,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Total,Jan
1,1,48 42.9 39,41 36.2 34,81 77.0 66,29 16.6 7,28.0 27.8 27.7,0.00,Jan
2,2,46 41.7 37,41 35.8 0,100 84.0 66,10 3.9 0,28.1 28.1 28.0,0.00,Jan
3,3,46 42.3 36,43 39.3 36,100 89.7 76,7 1.5 0,28.1 28.0 28.0,0.00,Jan
4,4,50 46.6 45,48 44.9 43,100 94.0 87,8 4.3 0,28.0 27.7 27.6,0.00,Jan
5,5,52 46.8 43,46 44.1 39,100 88.9 76,5 1.7 0,27.6 27.6 27.5,0.00,Jan
6,6,46 42.4 39,39 23.0 0,93 70.9 53,15 7.3 0,27.7 27.7 27.6,0.00,Jan
7,7,39 36.3 34,37 22.8 0,93 87.0 65,23 11.7 0,27.6 27.5 27.5,0.00,Jan
8,8,37 34.3 32,23 19.0 16,65 54.4 44,28 18.5 7,27.9 27.7 27.6,0.00,Jan
9,9,36 32.8 30,21 18.1 14,64 55.4 47,20 7.2 0,28.0 27.9 27.9,0.00,Jan


In [19]:
# separamos las columnas que tienen más de un dato 

clima_df[['Tmax','Tavg','Tmin']] = clima_df["Temperature (° F)"].str.split(" ",expand=True)

In [20]:
clima_df.head()

,Time,Temperature (° F),Dew Point (° F),Humidity (%),Wind Speed (mph),Pressure (Hg),Precipitation (in),month,Tmax,Tavg,Tmin
0,Jan,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Total,Jan,Max,Avg,Min
1,1,48 42.9 39,41 36.2 34,81 77.0 66,29 16.6 7,28.0 27.8 27.7,0.00,Jan,48,42.9,39
2,2,46 41.7 37,41 35.8 0,100 84.0 66,10 3.9 0,28.1 28.1 28.0,0.00,Jan,46,41.7,37
3,3,46 42.3 36,43 39.3 36,100 89.7 76,7 1.5 0,28.1 28.0 28.0,0.00,Jan,46,42.3,36
4,4,50 46.6 45,48 44.9 43,100 94.0 87,8 4.3 0,28.0 27.7 27.6,0.00,Jan,50,46.6,45


In [21]:
clima_df.loc[0]

,Time,Temperature (° F),Dew Point (° F),Humidity (%),Wind Speed (mph),Pressure (Hg),Precipitation (in),month,Tmax,Tavg,Tmin
0,Jan,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Total,Jan,Max,Avg,Min
0,Feb,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Total,Feb,Max,Avg,Min
0,Mar,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Max Avg Min,Total,Mar,Max,Avg,Min


In [22]:
# eliminamos cualquier fila que tenga un índice de 0

clima_df.drop([0], axis = 0,  inplace = True)

In [23]:
# volvemos a chequear los resultados

clima_df.head()

,Time,Temperature (° F),Dew Point (° F),Humidity (%),Wind Speed (mph),Pressure (Hg),Precipitation (in),month,Tmax,Tavg,Tmin
1,1,48 42.9 39,41 36.2 34,81 77.0 66,29 16.6 7,28.0 27.8 27.7,0.00,Jan,48,42.9,39
2,2,46 41.7 37,41 35.8 0,100 84.0 66,10 3.9 0,28.1 28.1 28.0,0.00,Jan,46,41.7,37
3,3,46 42.3 36,43 39.3 36,100 89.7 76,7 1.5 0,28.1 28.0 28.0,0.00,Jan,46,42.3,36
4,4,50 46.6 45,48 44.9 43,100 94.0 87,8 4.3 0,28.0 27.7 27.6,0.00,Jan,50,46.6,45
5,5,52 46.8 43,46 44.1 39,100 88.9 76,5 1.7 0,27.6 27.6 27.5,0.00,Jan,52,46.8,43
